# Battery Model
## Problem 
- Build an optimisation model to charge/discharge the battery over the time period provided 
(2018-2020) in order to maximise profits. You may assume that the battery is a price-taker (ie. the 
actions of the battery have no impact on the market prices).
- You can trade in the 3 markets. 
- The battery can export any amount of power up to its maximum discharge rate for any 
duration of time, as long as it has sufficient energy stored to do so (And same with the charging -- needs storage)
- Markets 1 and 2 are traded at half-hourly time granularity, whereas Market 3 is traded at daily 
granularity 
- The battery cannot sell the same unit of power into multiple markets, but can divide its power 
across the markets (The energy you give or take to the markets must add up to the total energy difference)
- For the battery to participate in Markets 1 and 2, it must export/import a constant level of 
power for the full half-hour period 
For the battery to participate in Market 3, it must export/import a constant level of power for 
the full day 
## Inputs
- Parameters are stored in the "config.yml" file
- Price data is stored in csv file - "data/input_data/market12_data.csv" and "data/input_data/market3_data.csv"
- Also there is information about the total production and different types of electricity production
## Outputs
- Total profits (Revenue - capex - opex*time)
- Data - Total energy in, Total energy out + any other profits 
- Plots for all optimisation variables - NICE TO HAVE
- Short description
- Dependencies
## Process
- I have decided to create 5 essential tasks 
1. Decide energy in and out from Market 1 with no change in capacity 
2. Decide energy in and out from Market 1 and 2 with no change in capacity
3. Decide energy in and out from Market 1 and 2 with change in capacity
4. Decide energy in and out from Market 1, 2 and 3 with change in capacity
5. Decide energy in and out from Market 1, 2 and 3 with change in capacity with difference in energy in and out efficiency. **(Explain later)**
6. Decide energy in and out from Market 1, 2 and 3 with change in capacity with weightings for different energy types.

## Variables
- $k$ is the sample number. Here it represents the number of half an hour steps the sample is from the start. 
- $m$ is the number of days from the start
- The prices $u_1[k]$, $u_2[k]$ and $u_3[m]$ represent the prices of each market at sample $k$. - market 1, 2 and 3 respectively
- The decision variables: 
- The energies into the battery at each sample ${x_{in}}_1[k]$, ${x_{in}}_2[k]$ and ${x_{in}}_3[m]$
- The energies into the battery at each sample ${x_{out}}_1[k]$, ${x_{out}}_2[k]$ and ${x_{out}}_3[m]$
- The net energy of the battery at each sample $E_{net}[k]$
- The number of cycles at each sample $N_{cyc}[k]$
- The maximum capacity of the battery at each sample $C_{max}[k]$
- We use a dummy variable to linearise the cycles model constraint $t[k]$
## Model
$$P_{in}[k]=\frac{\Sigma^2_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]}{\Delta t}$$
$$C_{max}[k+1]=C_{max}[k] - \mathtt{max \ storage}\times(\mathtt{degradation \ rate})^{N_{cyc}[k+1]-N_{cyc}[k]}\approx\mathtt{max \ storage}\times(1+ln(\mathtt{degradation \ rate}\times N_{cyc}[k]))$$
$$E_{in}[k+1] = E_{in}[k] +(\Sigma^2_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k])\times\mathtt{charging \ efficiency}$$
$$N_{cyc}[k+1]=N_{cyc}[k]+\frac{|\Sigma^2_{i=1} {x_{in}}_i[k] - {x_{out}}_i[k]|}{(2*C_{max}[k])}$$

## Assumptions
- The battery can discharge any amount of energy up to it's current energy
- The battery can charge any amount of energy up to it's current remaining storage
- The power is constant throughout a sample
- The efficiency of the charging is 95% -- therefore the battery only gets 95% of what it buys
- The efficiency of the discharging is 95% -- therefore the battery only recieves 95% of the money that what the energy discharged cost
- The battery can charge and discharge at the same time --> Energy in and energy out can both be positive
- **The battery maximum capacity does not change with time**
- **The battery can only use market 1 and 2**
## Initialisation
$$E_{net}[0]=0\mathtt{\ MW}\\
N_{cyc}[0]=0 \\ 
C_{max}=\mathtt{maximum \ storage \ volume\ MW}
$$

## Objective
- We want to maximise the profits by minimise the costs of the charging the batteries 
$$\begin{aligned}
\min_{{x_{in}}_1,{x_{out}}_1,{x_{in}}_2,{x_{out}}_2} \Sigma^2_{i=1} ({x_{in}}_i[k] - {x_{out}}_i[k])\times u_i[k]
\end{aligned}$$

## Constraints
- Additional to the model constaints
- Power is limited to the maximum charging and discharging rate 
$$ \frac{\Sigma x_in[k]}{\mathtt{number \ of \ hours}}  <= \mathtt{max \ charging \ rate} $$
$$ \frac{\Sigma x_out[k]}{\mathtt{number \ of \ hours}}  <= \mathtt{max \ discharging \ rate} $$
- The number of cycles is a parameter that must be exceeded
$$ 0 <= N_{cyc}[k] <= \mathtt{max \ cycles}$$
- The number of years is also limited
$$ 0 <= k / 31,536,000 <= \mathtt{max \ cycles}$$
- The energy is limited by its maximum capacity
$$ 0 <= E_{net}[k] <= C_{max}[k]$$
## Solver choice 
- Since the problem is linear. I chose the solver - Coin branch and cut
- Also there is an assertion to check that the next energy is equal to this energy + $\Delta energy$ 
## References 
- "Introduction to Operations Research" by Frederick S. Hillier and Gerald J. Lieberman



In [1]:
using IJulia, CSV, DataFrames, JuMP, YAML, Infiltrator, Dates
include("batterymodel.jl")
include("plot_funcs.jl")

input_data = CSV.read("data/input_data/market12_data.csv", DataFrame)
config_data = YAML.load_file("config.yml")

marketprices1 = input_data.Market_1
marketprices2 = input_data.Market_2
start_point = DateTime.(input_data.time[1], "dd/mm/yyyy HH:MM")
params = BatteryModel.BatteryParams(
            config_data["max_charging_rate"],
            config_data["max_discharging_rate"],
            config_data["max_storage_volume"],
            config_data["charging_efficiency"],
            config_data["discharging_efficiency"],
            config_data["lifetime_years"],
            config_data["lifetime_cycles"],
            config_data["degradation_rate"],
            config_data["capex"], 
            config_data["opex"]
)

N = length(marketprices1)
marketprices = hcat(marketprices1, marketprices2)
marketprices = marketprices.*0.5
# marketprices = marketprices[1:Int(round(N/100)),:]
print("market prices: $marketprices")


LoadError: InterruptException:

In [ ]:
energy_in, energy_out, energies, cycle, maximum_capacities, powers = BatteryModel.optimise_battery_charge(marketprices, params)



In [ ]:
PlotFuncs.plot_battery_performance(marketprices, energy_in, energy_out, energies, powers, cycle, maximum_capacities)

PlotFuncs.write_battery_performance(marketprices, energy_in, energy_out, energies, cycle, maximum_capacities, start_point, params)

"data/output_data/battery_output_data.cvs"